In [1]:
import os
import torch
import cv2
import numpy as np
import tqdm
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
Rebuild_data = True
class pet_classification:
    def __init__(self):
        self.cats = 'kagglecatsanddogs_5340\PetImages\Cat'
        self.dogs = 'kagglecatsanddogs_5340\PetImages\Dog'
        self.labels = {self.cats:0,self.dogs:1}
        self.catcount = 0
        self.dogcount = 0
        self.training_data = []

    def create_Dataset(self):
        for label in self.labels:
            print(label)
            for file in tqdm.tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label,file)
                    image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image,(50,50))
                    self.training_data.append([np.array(image),np.eye(2)[self.labels[label]]])
                    
                    if label == self.cats:
                        self.catcount += 1
                    elif label == self.dogs:
                        self.dogcount += 1
                
                except Exception as e:
                    # print(str(e))
                    pass
            
        print(f"Cats: {self.catcount} and Dogs: {self.dogcount}")
        np.random.shuffle(self.training_data)
        with open('training_data.npy','wb') as file:
            pickle.dump(self.training_data,file)
        return self.training_data




In [29]:
something = pet_classification()
training_Data = something.create_Dataset()

kagglecatsanddogs_5340\PetImages\Cat


100%|██████████| 12501/12501 [00:08<00:00, 1478.20it/s]


kagglecatsanddogs_5340\PetImages\Dog


100%|██████████| 12501/12501 [00:09<00:00, 1372.93it/s]


Cats: 12476 and Dogs: 12470


In [3]:
training_data = np.load('training_data.npy',allow_pickle=True)

In [4]:
print(len(training_data))

24946


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512,512)
        self.fc2 = nn.Linear(512,2)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        return(x)
    

In [6]:
model = CNN()
something = torch.randn(1, 1, 50, 50)
output = model(something)
print(output)

tensor([[0.4821, 0.5179]], grad_fn=<SoftmaxBackward0>)


In [7]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_function = nn.MSELoss()


In [8]:
X = torch.tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255
y = torch.tensor([i[1] for i in training_data])


C:\Users\tejas\AppData\Local\Temp\ipykernel_16584\3621654422.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  X = torch.tensor([i[0] for i in training_data]).view(-1,50,50)


In [9]:
len(X)

24946

In [10]:
test_size = int(len(X)*0.2)
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model = model.to(device)
print(device)

cuda


In [11]:
train_x = X[:-test_size].to(device)
train_y = y[:-test_size].to(device)

test_x = X[-test_size:].to(device)
test_y = y[-test_size:].to(device)

print(len(train_x), len(test_x))
print(len(train_y), len(test_y))

19957 4989
19957 4989


In [12]:
batch_size = 32
num_epochs = 5

In [19]:
for epoch in range(num_epochs):
    for i in tqdm.tqdm(range(0,len(train_x),batch_size)):
        batch_x = train_x[i:i+batch_size].view(-1,1,50,50).to(device)
        batch_y = train_y[i:i+batch_size].to(device).float()
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_function(outputs,batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, loss: {loss.item}')


100%|██████████| 624/624 [00:02<00:00, 292.03it/s]


Epoch: 0, loss: <built-in method item of Tensor object at 0x000001D987B11C70>


100%|██████████| 624/624 [00:02<00:00, 293.27it/s]


Epoch: 1, loss: <built-in method item of Tensor object at 0x000001D9876E5AE0>


100%|██████████| 624/624 [00:02<00:00, 282.88it/s]


Epoch: 2, loss: <built-in method item of Tensor object at 0x000001D987CC8450>


100%|██████████| 624/624 [00:02<00:00, 279.49it/s]


Epoch: 3, loss: <built-in method item of Tensor object at 0x000001D987A19130>


100%|██████████| 624/624 [00:02<00:00, 269.88it/s]

Epoch: 4, loss: <built-in method item of Tensor object at 0x000001D986FF99A0>


In [14]:
total = 0
correct = 0
with torch.no_grad():
    for i in tqdm.tqdm(range(len(test_x))):
        real_class = torch.argmax(test_y[i])
        outputs = model(test_x[i].view(-1,1,50,50))[0]
        predictions = torch.argmax(outputs)
        if predictions==real_class:
            correct +=  1
        total+=1

print(f'Accuracy: {correct/total}')

100%|██████████| 4989/4989 [00:06<00:00, 739.46it/s] 

Accuracy: 0.7518540789737422


In [59]:
with open('cat_dog_classifier','wb') as file:
    pickle.dump(model,file)

In [13]:
with open('cat_dog_classifier','rb') as file:
    model = pickle.load(file)

In [61]:
image_path = 'dog_image.jpg'
image = cv2.imread(image_path)
image = image.reshape(50,50,-1)
image_tensor = torch.tensor(image)
image_tensor = image_tensor.to(device)
image_tensor = image_tensor.float()
image_tensor = image_tensor.view(-1,1,50,50)

In [63]:
output = model(image_tensor)
prediction = 'cat' if torch.argmax(output)==0 else 'dog'
print(prediction)

dog
